In [2]:
import sys
import os
import shutil
import subprocess
from PyQt5.QtCore import Qt, pyqtSignal
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QTextEdit, QVBoxLayout, QHBoxLayout, QWidget
from PyQt5.QtGui import QPixmap, QDragEnterEvent, QDropEvent

class ImageProcessorGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.raw_images_folder = 'Raw Images'
        self.output_folder = 'Numbered Images'
        self.unusable_folder = 'Unusable Images'
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Image Processor')
        self.setGeometry(100, 100, 800, 600)

        # Create widgets
        self.rawImageLabel = DropLabel('Drop Raw Image Here')
        self.rawImageLabel.setAlignment(Qt.AlignCenter)
        self.rawImageLabel.setStyleSheet('border: 2px dashed #aaa')
        self.rawImageLabel.setAcceptDrops(True)
        self.rawImageLabel.dropSignal.connect(self.handleDroppedImage)

        self.processedImageLabel = QLabel('Processed Image')
        self.processedImageLabel.setAlignment(Qt.AlignCenter)
        self.processedImageLabel.setStyleSheet('border: 2px dashed #aaa')

        self.processButton = QPushButton('Process')
        self.processButton.clicked.connect(self.processImage)
        self.processButton.setEnabled(False)

        self.logTextEdit = QTextEdit()
        self.logTextEdit.setReadOnly(True)

        # Layout
        mainLayout = QVBoxLayout()
        imageLayout = QHBoxLayout()
        imageLayout.addWidget(self.rawImageLabel)
        imageLayout.addWidget(self.processedImageLabel)
        mainLayout.addLayout(imageLayout)
        mainLayout.addWidget(self.processButton)
        mainLayout.addWidget(self.logTextEdit)

        centralWidget = QWidget()
        centralWidget.setLayout(mainLayout)
        self.setCentralWidget(centralWidget)

        self.current_image_path = None

    def handleDroppedImage(self, file_path):
        file_name = os.path.basename(file_path)
        destination = os.path.join(self.raw_images_folder, file_name)
        shutil.copy2(file_path, destination)
        self.logTextEdit.append(f'Copied {file_name} to Raw Images folder')
        self.updateRawImagePreview(destination)
        self.current_image_path = destination
        self.processButton.setEnabled(True)

    def updateRawImagePreview(self, file_path):
        pixmap = QPixmap(file_path)
        scaled_pixmap = pixmap.scaled(self.rawImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.rawImageLabel.setPixmap(scaled_pixmap)

    def processImage(self):
        if not self.current_image_path:
            self.logTextEdit.append('No image to process')
            return

        self.logTextEdit.append('Processing image...')
        try:
            # Run the Jupyter notebook script
            result = subprocess.run(['python', '-m', 'nbconvert', '--to', 'script', 'inference_numbering.ipynb', '--output', 'temp_script.py'], 
                                    capture_output=True, text=True)
            if result.returncode != 0:
                raise Exception(f"Failed to convert notebook: {result.stderr}")

            result = subprocess.run(['python', 'temp_script.py'], capture_output=True, text=True)
            if result.returncode != 0:
                raise Exception(f"Failed to run script: {result.stderr}")

            self.logTextEdit.append('Processing complete')
            self.updateProcessedImagePreview()
            os.remove(self.current_image_path)
            self.logTextEdit.append(f"Deleted original image: {os.path.basename(self.current_image_path)}")
            self.current_image_path = None
            self.processButton.setEnabled(False)
            self.rawImageLabel.clear()
            self.rawImageLabel.setText('Drop Raw Image Here')

        except Exception as e:
            self.logTextEdit.append(f"Error processing image: {str(e)}")

    def updateProcessedImagePreview(self):
        for folder in [self.output_folder, self.unusable_folder]:
            for file_name in os.listdir(folder):
                if file_name.startswith(('numbered_', 'unusable_')):
                    file_path = os.path.join(folder, file_name)
                    pixmap = QPixmap(file_path)
                    scaled_pixmap = pixmap.scaled(self.processedImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    self.processedImageLabel.setPixmap(scaled_pixmap)
                    return
        self.logTextEdit.append('No processed image found')

class DropLabel(QLabel):
    dropSignal = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setAcceptDrops(True)

    def dragEnterEvent(self, event: QDragEnterEvent):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event: QDropEvent):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            self.dropSignal.emit(file_path)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageProcessorGUI()
    ex.show()
    sys.exit(app.exec_())

QSocketNotifier: Can only be used with threads started with QThread


In [1]:
import sys
import os
import shutil
import subprocess
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QTextEdit, QVBoxLayout, QHBoxLayout, QWidget
from PyQt5.QtCore import Qt, pyqtSignal
from PyQt5.QtGui import QPixmap, QDragEnterEvent, QDropEvent

class DropLabel(QLabel):
    dropSignal = pyqtSignal(str)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.setAcceptDrops(True)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()

    def dragMoveEvent(self, event):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            self.dropSignal.emit(file_path)

class ImageProcessorGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.raw_images_folder = 'Raw Images'
        self.output_folder = 'Numbered Images'
        self.unusable_folder = 'Unusable Images'
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Image Processor')
        self.setGeometry(100, 100, 1200, 800)  # Increased window size

        # Create widgets
        self.rawImageLabel = DropLabel('Drop Raw Image Here')
        self.rawImageLabel.setAlignment(Qt.AlignCenter)
        self.rawImageLabel.setStyleSheet('border: 2px dashed #aaa; font-size: 20px;')
        self.rawImageLabel.setMinimumSize(500, 500)  # Set minimum size
        self.rawImageLabel.dropSignal.connect(self.handleDroppedImage)

        self.processedImageLabel = QLabel('Processed Image')
        self.processedImageLabel.setAlignment(Qt.AlignCenter)
        self.processedImageLabel.setStyleSheet('border: 2px dashed #aaa; font-size: 20px;')
        self.processedImageLabel.setMinimumSize(500, 500)  # Set minimum size

        self.processButton = QPushButton('Process')
        self.processButton.clicked.connect(self.processImage)
        self.processButton.setEnabled(False)
        self.processButton.setStyleSheet('font-size: 16px; padding: 10px;')

        self.logTextEdit = QTextEdit()
        self.logTextEdit.setReadOnly(True)
        self.logTextEdit.setStyleSheet('font-size: 14px;')

        # Layout
        mainLayout = QVBoxLayout()
        imageLayout = QHBoxLayout()
        imageLayout.addWidget(self.rawImageLabel, 1)
        imageLayout.addWidget(self.processedImageLabel, 1)
        mainLayout.addLayout(imageLayout, 4)
        mainLayout.addWidget(self.processButton)
        mainLayout.addWidget(self.logTextEdit, 1)

        centralWidget = QWidget()
        centralWidget.setLayout(mainLayout)
        self.setCentralWidget(centralWidget)

        self.current_image_path = None

    def handleDroppedImage(self, file_path):
        file_name = os.path.basename(file_path)
        destination = os.path.join(self.raw_images_folder, file_name)
        os.makedirs(self.raw_images_folder, exist_ok=True)
        shutil.copy2(file_path, destination)
        self.logTextEdit.append(f'Copied {file_name} to Raw Images folder')
        self.updateRawImagePreview(destination)
        self.current_image_path = destination
        self.processButton.setEnabled(True)

    def updateRawImagePreview(self, file_path):
        pixmap = QPixmap(file_path)
        scaled_pixmap = pixmap.scaled(self.rawImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.rawImageLabel.setPixmap(scaled_pixmap)

    def processImage(self):
        if not self.current_image_path:
            self.logTextEdit.append('No image to process')
            return

        self.logTextEdit.append('Processing image...')
        try:
            # Run the Jupyter notebook script
            result = subprocess.run(['python3', '-m', 'nbconvert', '--to', 'script', 'inference_numbering.ipynb', '--output', 'temp_script.py'], 
                                    capture_output=True, text=True)
            if result.returncode != 0:
                raise Exception(f"Failed to convert notebook: {result.stderr}")

            result = subprocess.run(['python3', 'temp_script.py'], capture_output=True, text=True)
            if result.returncode != 0:
                raise Exception(f"Failed to run script: {result.stderr}")

            self.logTextEdit.append('Processing complete')
            self.updateProcessedImagePreview()
            os.remove(self.current_image_path)
            self.logTextEdit.append(f"Deleted original image: {os.path.basename(self.current_image_path)}")
            self.current_image_path = None
            self.processButton.setEnabled(False)
            self.rawImageLabel.clear()
            self.rawImageLabel.setText('Drop Raw Image Here')

        except Exception as e:
            self.logTextEdit.append(f"Error processing image: {str(e)}")


    def updateProcessedImagePreview(self):
        for folder in [self.output_folder, self.unusable_folder]:
            for file_name in os.listdir(folder):
                if file_name.startswith(('numbered_', 'unusable_')):
                    file_path = os.path.join(folder, file_name)
                    pixmap = QPixmap(file_path)
                    scaled_pixmap = pixmap.scaled(self.processedImageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    self.processedImageLabel.setPixmap(scaled_pixmap)
                    return
        self.logTextEdit.append('No processed image found')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageProcessorGUI()
    ex.show()
    sys.exit(app.exec_())

QSocketNotifier: Can only be used with threads started with QThread
